In [3]:
from shapely.geometry import Point
from shapely.strtree import STRtree

from litecoder.models import Locality

In [13]:
points = [Point(i, i) for i in range(10000)]

In [14]:
tree = STRtree(points)

In [16]:
%time tree.query(Point(3,3).buffer(2))

CPU times: user 641 µs, sys: 457 µs, total: 1.1 ms
Wall time: 559 µs


In [17]:
Point?

Init signature: Point(*args)
Docstring:     
A zero dimensional feature

A point has zero length and zero area.

Attributes
----------
x, y, z : float
    Coordinate values

Example
-------
  >>> p = Point(1.0, -1.0)
  >>> print(p)
  POINT (1.0000000000000000 -1.0000000000000000)
  >>> p.y
  -1.0
  >>> p.x
  1.0
Init docstring:
Parameters
----------
There are 2 cases:

1) 1 parameter: this must satisfy the numpy array protocol.
2) 2 or more parameters: x, y, z : float
    Easting, northing, and elevation.
File:           ~/Projects/litecoder/env/lib/python3.6/site-packages/shapely/geometry/point.py
Type:           type
